### Load imports and run the no LID case:

In [2]:
%reset
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import runswmm
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys
#client = MongoClient('mongodb://server.mcgarity.info:27017/')
client = MongoClient()
db = client.swmmwiseDebug
runsToday = db.y16m01d23
swmmInpFile = "Example2_LID_BR_for_RG_PHL.inp"
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
f = open('runlist_start.yaml','r')   # Read the no LID changable parameters from file
noLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
noLidDocId = runswmm(noLidList,swmmInpStr,runsToday)  # run SWMM on the no LID model for baseline
cursor = runsToday.find_one({'_id':noLidDocId})
noLidPeak = cursor['peak']
noLidVolume = cursor['volume']
print "Stored record %s for no LID parameters, peak = %s, volume = %s" % (noLidDocId,noLidPeak,noLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
0.0 min, 11.70 sec
Stored record 56a3bc16c10157a54bf3908c for no LID parameters, peak = 21.52, volume = 15.563


### Change ALL subcatchment LID numbers:

In [4]:
numberForAllLid = 1
newRunList=list(noLidList)
for lid in newRunList:
    lid['Number'] = numberForAllLid
newRunDocId = runswmm(noLidList,swmmInpStr,runsToday)
cursor = runsToday.find_one({'_id':newRunDocId})
newPeak = cursor['peak']
newVolume = cursor['volume']
peakReduction = noLidPeak - newPeak
volumeReduction = noLidVolume - newVolume
print "Stored record %s for new parameters: peak = %s, volume = %s" % (newRunDocId,newPeak,newVolume)
print "peakReduction = %s, volumeReduction = %s" % (peakReduction,volumeReduction)

0.0 min, 13.13 sec
Stored record 56a3bd1cc10157a54bf3908d for new parameters: peak = 21.5, volume = 15.109
peakReduction = 0.02, volumeReduction = 0.454


In [5]:
runs = db.y16m01d23
cursor = runs.find_one({'_id':noLidDocId})
print cursor
print ''
cursor = runs.find_one({'_id':newRunDocId})
print cursor

{u'lidDict': None, u'swmmStartTime': u'2016-01-23 12:44:42.868904', u'volume': 15.563, u'peak': 21.52, u'runParamList': [{u'LID': u'wakefield_BR_RG', u'Subcat': u'S1', u'Area': 741, u'Number': 0, u'Width': 0, u'FromImp': 100}], u'_id': ObjectId('56a3bc16c10157a54bf3908c'), u'swmmRunTime': u'0.0 min, 11.70 sec'}

{u'lidDict': {u'S1 wakefield_BR_RG': {u'Infil Loss': 966.44, u'Total Inflow': 5250.33, u'Evap Loss': 21.2, u'Surface Outflow': 4262.7, u'Continuity Error': -0.0, u'Initial Storage': 0.3, u'Final Storage': 0.5, u'Drain Outflow': 0.0}}, u'swmmStartTime': u'2016-01-23 12:49:03.211270', u'volume': 15.109, u'peak': 21.5, u'runParamList': [{u'LID': u'wakefield_BR_RG', u'Subcat': u'S1', u'Area': 741, u'Number': 1, u'Width': 0, u'FromImp': 100}], u'_id': ObjectId('56a3bd1cc10157a54bf3908d'), u'swmmRunTime': u'0.0 min, 13.13 sec'}
